<a href="https://colab.research.google.com/github/HYAD-Yassin/Password_Factory/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Premiere Base de Données**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!ls drive/MyDrive/

Mounted at /content/drive
 100K_LSTM_passwords.txt
 100K_RNN_Passwords.txt
 10K_LSTM_passwords.txt
 1M_LSTM_passwords.txt
 acoustique_voy_orales_20loc_ESTER_NCCFr_contexte_freqLex_distCentroide.csv
 AllDataSet_Filtred.txt
 AllDataSet.txt
'Alternance (1).gdoc'
 Alternance.gdoc
 archive.zip
 Ashley-Madison_Ini.txt
 Ashley-Madison.txt
'Colab Notebooks'
 DATABASE_Password.zip
 data.zip
 Filtered-Ashley-Madison.txt
 Filtered_PWD.txt
 generated100k_GRU_passwords.txt
 generated100K_LSTM_passwords.txt
 generated10K_LSTM_passwords.txt
 generated1M_GRU_passwords.txt
 generated200K_LSTM_passwords.txt
 generated50K_LSTM_passwords.txt
 generated_GRU_passwords.txt
 generated_LSTM_passwords.txt
 generated_passwords.txt
 gru_model.pth
 histo2.png
 histogram1.png
 Letter.gdoc
 lstm_model2.pth
 lstm_model.pth
 my_model
 nameGeneration.py
 Passwords.txt
 pwd_2Rnn.pth
 pwd_Rnn.pth
 __pycache__
 reduced_Ashley-Madison.txt
 rnn.pt
'Untitled document (1).gdoc'
'Untitled document.gdoc'


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
from sklearn.model_selection import train_test_split

# Read the file and prepare the data
file_path = '/content/drive/My Drive/Ashley-Madison.txt'
with open(file_path, 'r') as file:
    data = file.read()

# Create a mapping of characters to integers
chars = sorted(list(set(data)))
char_to_int = {c: i for i, c in enumerate(chars)}
int_to_char = {i: c for i, c in enumerate(chars)}
n_characters = len(chars)  # Number of unique characters

# Prepare the dataset with integer encoding
max_length = 100  # Maximum length of a password
step = 3  # Step size for sampling

sequences = []
for i in range(0, len(data) - max_length, step):
    sequences.append(data[i: i + max_length])

# Split the data into training (70%) and validation (30%) sets
train_sequences, _ = train_test_split(sequences, train_size=0.7, random_state=42)

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


#**Dataset**

In [ ]:
import re

# The path to your file - replace with the actual path
file_path = '/content/drive/My Drive/Ashley-Madison.txt'

# Function to filter passwords
def filter_passwords(text):
    # Splitting text into lines assuming one password per line
    lines = text.split('\n')
    # Filtering lines where length of line is between 6 and 15 characters
    filtered_lines = [line for line in lines if 6 <= len(line) <= 15]
    return '\n'.join(filtered_lines)

# Read the file
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

# Filter the text
filtered_text = filter_passwords(text)

# Write the filtered text back to the file
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(filtered_text)

FileNotFoundError: ignored

#**RNN**

In [ ]:
# RNN Class
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, n_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 256)
        self.fc2 = nn.Linear(256, output_size)


    def forward(self, input, hidden):
        # Embedding layer expects input of shape [batch_size, seq_len] with batch_size=1
        embedded = self.embedding(input.view(1, -1))
        output, hidden = self.rnn(embedded, hidden)
        output=self.fc1(output)
        output = self.fc2(output.view(1, -1))
        return output

    def initHidden(self):
        return torch.zeros(self.n_layers, 1, self.hidden_size)

# Convert string to tensor
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = char_to_int[string[c]]
    return tensor.to(device)

# Random Training Example
def random_training_example():
    line = random.choice(train_sequences)
    input_tensor = char_tensor(line[:-1])
    target_tensor = char_tensor(line[1:])
    return input_tensor, target_tensor

# Initialize the RNN
hidden_size = 512
rnn = RNN(n_characters, hidden_size, n_characters).to(device)

# Training
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), lr=1e-3)

# Training function with device support
def train(input_tensor, target_tensor):
    hidden = rnn.initHidden().to(device)  # Move hidden state to the device

    rnn.zero_grad()
    loss = 0

    input_tensor = input_tensor.to(device)  # Move input tensor to the device
    target_tensor = target_tensor.to(device)  # Move target tensor to the device

    for i in range(input_tensor.size(0)):
        output = rnn(input_tensor[i].unsqueeze(0), hidden)
        l = criterion(output, target_tensor[i].unsqueeze(0))
        loss += l

    loss.backward()
    optimizer.step()

    return loss.item() / input_tensor.size(0)

**Training**

In [ ]:
# Training Loop
n_epochs = 5000
print_every = 500
current_loss = 0

for epoch in range(1, n_epochs + 1):
    input_tensor, target_tensor = random_training_example()
    loss = train(input_tensor, target_tensor)
    current_loss += loss

    if epoch % print_every == 0:
        print(f'Epoch {epoch} Loss: {current_loss / print_every:.4f}')
        current_loss = 0


Epoch 500 Loss: 3.2301
Epoch 1000 Loss: 3.1195
Epoch 1500 Loss: 3.0833
Epoch 2000 Loss: 3.0896
Epoch 2500 Loss: 3.0809
Epoch 3000 Loss: 3.0759
Epoch 3500 Loss: 3.0705
Epoch 4000 Loss: 3.0787
Epoch 4500 Loss: 3.0720
Epoch 5000 Loss: 3.0676


In [ ]:
model_save_path = '/content/drive/My Drive/pwd_2Rnn.pth'  # Modify as needed
torch.save(rnn.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

NameError: ignored

In [ ]:
# Initialize model
rnn = RNN(n_characters, hidden_size, n_characters).to(device)

# Mount Google Drive (specific to Google Colab, you might not need this if your environment already has access to the drive)
from google.colab import drive
drive.mount('/content/drive')

# Path to your model file
model_save_path = '/content/drive/My Drive/pwd_2Rnn.pth'

# Load the model onto CPU
rnn.load_state_dict(torch.load(model_save_path, map_location=torch.device('cpu')))
print(f"Model loaded from {model_save_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model loaded from /content/drive/My Drive/pwd_2Rnn.pth


**Passwords Generation**

In [ ]:
def generate_password(model, start_str='A', predict_len=100, temperature=0.8):
    model.to(device)  # Ensure the model is on the right device
    hidden = model.initHidden().to(device)
    input_tensor = char_tensor(start_str).to(device)
    predicted_str = start_str

    for p in range(len(start_str) - 1):
        _, hidden = model(input_tensor[p].unsqueeze(0), hidden)

    last_char = input_tensor[-1]

    for p in range(predict_len):
        output = model(last_char.unsqueeze(0), hidden)
        output_dist = output.data.view(-1).div(temperature).exp()
        top_char = torch.multinomial(output_dist, 1)[0]

        predicted_char = int_to_char[top_char.item()]
        predicted_str += predicted_char
        last_char = char_tensor(predicted_char)

    return predicted_str

def generate_passwords_to_file(model, num_pwd, file_path, start_str='A', predict_len=100, temperature=0.85):
    with open(file_path, 'w') as file:
        for _ in range(num_pwd):
            password = generate_password(model, start_str, predict_len, temperature)
            file.write(password + '\n')

# Example usage: Generate a specified number of passwords and save them to a file
num_pwd = 100000  # Specify the number of passwords to generate
output_file_path = '1M_RNN_passwords.txt'  # Modify this path as needed
generate_passwords_to_file(rnn, num_pwd, output_file_path)


In [ ]:
!cp '1M_RNN_passwords.txt' '/content/drive/My Drive'

**Calculate Accuracy**

In [ ]:
def calculate_accuracy_and_matches(original_dataset_path, generated_file_path):
    # Load the original dataset
    with open(original_dataset_path, 'r') as file:
        original_passwords = set(file.read().splitlines())

    # Load generated passwords
    with open(generated_file_path, 'r') as file:
        generated_passwords = file.read().splitlines()

    # Find matches
    matches = [password for password in generated_passwords if password in original_passwords]
    # Calculate accuracy based on full line matches
    accuracy = (len(matches) / len(original_passwords)) * 100 if generated_passwords else 0
    return accuracy, matches

# Paths to the files
original_dataset_path = '/content/drive/My Drive/Ashley-Madison_Ini.txt'  # Modify as needed
generated_file_path = '/content/drive/My Drive/generated_GRU_passwords.txt'  # Modify as needed

# Calculate accuracy and get matches
accuracy, matching_passwords = calculate_accuracy_and_matches(original_dataset_path, generated_file_path)
print(f"Accuracy: {accuracy:.2f}%  Matches / DataSet Initial")

print("Matching Passwords Number:", len(matching_passwords))
print("Matching Passwords:", set(matching_passwords))

Accuracy: 0.00%  Matches / DataSet Initial
Matching Passwords Number: 1
Matching Passwords: {'020300c'}


In [ ]:
# Read the data
with open(generated_file_path, 'r', encoding='utf-8') as file:
    passwords = file.read().splitlines()  # Each password is a line

print(f"The total passwords in The DataSet is: {len(passwords)}")

The total passwords in The DataSet is: 10000
